In [1]:
import pandas as pd
import json
from urllib.request import Request, urlopen

import numpy as np
import tls_client
import pandas as pd
import webbrowser
import requests
import re
import plotly.express as px
import seaborn as sns
from utils2 import stats_dict, remove_NFLSZN_ud, remove_NFLSZN_pp# merge_books, pp_delta, ud_delta

pd.set_option('display.max_columns', None)

In [2]:
# NFL API data

import nfl_data_py as nfl

years = [2024]
nfl_df = nfl.import_weekly_data(years,downcast=True)

nfl_df.head()

Downcasting floats.


,player_id,player_name,player_display_name,position,position_group,headshot_url,recent_team,season,week,season_type,opponent_team,completions,attempts,passing_yards,passing_tds,interceptions,sacks,sack_yards,sack_fumbles,sack_fumbles_lost,passing_air_yards,passing_yards_after_catch,passing_first_downs,passing_epa,passing_2pt_conversions,pacr,dakota,carries,rushing_yards,rushing_tds,rushing_fumbles,rushing_fumbles_lost,rushing_first_downs,rushing_epa,rushing_2pt_conversions,receptions,targets,receiving_yards,receiving_tds,receiving_fumbles,receiving_fumbles_lost,receiving_air_yards,receiving_yards_after_catch,receiving_first_downs,receiving_epa,receiving_2pt_conversions,racr,target_share,air_yards_share,wopr,special_teams_tds,fantasy_points,fantasy_points_ppr
0,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,1,REG,SF,13,21,167.0,1,1.0,1.0,5.0,0,0,164.0,93.0,8.0,3.258283,0,1.018293,0.066574,1,-1.0,0,0.0,0.0,0.0,0.000000,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,8.580000,8.580000
1,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,2,REG,TEN,18,30,176.0,2,0.0,2.0,12.0,0,0,167.0,96.0,9.0,1.653769,0,1.053892,0.040780,2,1.0,0,0.0,0.0,0.0,-0.327141,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,15.140000,15.140000
2,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,3,REG,NE,27,35,281.0,2,0.0,2.0,14.0,0,0,226.0,152.0,16.0,18.204784,0,1.243363,0.257863,3,18.0,0,0.0,0.0,2.0,1.414984,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,21.040001,21.040001
3,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,4,REG,DEN,24,42,225.0,0,0.0,5.0,41.0,0,0,306.0,96.0,10.0,-12.087543,0,0.735294,-0.016657,5,26.0,0,0.0,0.0,1.0,-0.570366,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.600000,11.600000
4,00-0023459,A.Rodgers,Aaron Rodgers,QB,QB,https://static.www.nfl.com/image/upload/f_auto...,NYJ,2024,5,REG,MIN,29,54,244.0,2,3.0,3.0,26.0,1,0,410.0,123.0,13.0,-16.156328,0,0.595122,-0.019010,0,0.0,0,0.0,0.0,0.0,NaN,0,0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,NaN,0.0,11.760000,11.760000


In [ ]:
# UNDERDOG API

def getUnderdog(sportName2=None):
        url = 'https://api.underdogfantasy.com/beta/v3/over_under_lines'
        req2 = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        dataFile2 = json.loads(urlopen(req2).read())
        underdogFantasy = []
        for dataNames in dataFile2['over_under_lines']:
                splitted = dataNames['over_under']['title'].split()
                underdogFantasy.append({
                        'OverUnder': dataNames['stat_value'],
                        'Name': splitted[0]+" "+splitted[1],
                        'Stat Type': dataNames['over_under']['appearance_stat']['display_stat']
                })
                names2 = []
        for dataNames in dataFile2['players']:
                names2.append({
                        'First Name': dataNames['first_name'],
                        'Last Name': dataNames['last_name'],
                        'Sport': dataNames['sport_id']
                })

        nameFrame = pd.DataFrame(names2)
        nameFrame = nameFrame[nameFrame['Sport'] != 'NFLSZN']
        underdogFrame = pd.DataFrame(underdogFantasy)
        nameFrame['Name'] = nameFrame['First Name'] + " " + nameFrame['Last Name']

        result = pd.merge(underdogFrame, nameFrame, on=["Name"])
        result = result[['Name', 'Stat Type', 'OverUnder', 'Sport']]

        if sportName2 == None:
                return result
        else:
                sportName2 = str(sportName2).upper()
                result = result.loc[result['Sport'] == sportName2]
                return result
        
ud = getUnderdog('NFL')

ud = ud.rename(columns={'Name':'player','Stat Type':'market','OverUnder':'ud_line'})
ud = ud.drop(columns=['Sport'])

ud['market'] = ud['market'].map(stats_dict)                             # normalize market column
ud = ud.astype({'player':str,'market':'category','ud_line':float})      # convert to string and category
ud = remove_NFLSZN_ud(ud)                                               # remove season long props

ud.head()
        

,player,market,ud_line
680,Jordan Love,passing_yards,252.50
681,Jordan Love,pass+rush_yards,261.50
682,Jordan Love,rushing_yards,5.50
683,Jordan Love,fantasy_points,18.05
684,Jordan Love,longest_rush,5.50


In [ ]:
# def remove_NFLSZN_pp(df):
#     # Filter out rows based on the conditions
#     filtered_df = df[~(
#         ((df['market'] == 'passing_yards') & (df['pp_line'] > 500)) |
#         ((df['market'] == 'receiving_yards') & (df['pp_line'] > 200)) |
#         ((df['market'] == 'receiving_tds') & (df['pp_line'] > 2.5)) |
#         ((df['market'] == 'receptions') & (df['pp_line'] > 15)) |
#         ((df['market'] == 'rushing_tds') & (df['pp_line'] > 3)) |
#         ((df['market'] == 'rushing_yards') & (df['pp_line'] > 200)) |
#         ((df['market'] == 'carries') & (df['pp_line'] > 30))
#     )]
#     return filtered_df

,player,market,pp_line
1,Jordan Love,completions,21.0
2,Jahmyr Gibbs,rushing_yards,67.5
3,Amon-Ra St. Brown,receiving_yards,72.5
5,Josh Jacobs,rush+rec_yards,90.5
8,Jordan Love,attempts,33.0


In [7]:
# get bet rivers lines
br = pd.read_csv(r"data\br_lines.csv").rename(columns={'line':'br_line'})
br.head()

,player,market,br_line
0,Jared Goff,passing_yards,252.5
1,Jordan Love,passing_yards,249.5
2,Sam LaPorta,receiving_yards,34.5
3,Amon-Ra St. Brown,receiving_yards,94.5
4,Christian Watson,receiving_yards,34.5


In [8]:
def merge_books(ud, pp, br, nfl_df):
    # Merge df1 and df2 on 'player' and 'market' with an outer join
    merged_df = pd.merge(ud, pp, on=['player', 'market'], how='outer')
    
    # Merge the result with df3 on 'player' and 'market' with an outer join
    merged_df = pd.merge(merged_df, br, on=['player', 'market'], how='outer')

    # Merge in NFL API data
    merged_df = merged_df.merge(nfl_df,left_on='player',right_on='player_display_name',how='left').rename(columns={'recent_team':'team'})
    
    return merged_df

merged_books = merge_books(ud, pp, br, nfl_df)[['player','market','ud_line','pp_line','br_line',
                                                'team','opponent_team','week',   
                                                'completions', 'attempts','passing_yards', 'passing_tds',    
                                                'carries', 'rushing_yards','rushing_tds', 'receptions',      
                                                'targets','receiving_yards', 'receiving_tds', 'target_share',
                                                'air_yards_share','fantasy_points', 'fantasy_points_ppr',    
                                                'headshot_url']]
merged_books.head(2)

,player,market,ud_line,pp_line,br_line,team,opponent_team,week,completions,attempts,passing_yards,passing_tds,carries,rushing_yards,rushing_tds,receptions,targets,receiving_yards,receiving_tds,target_share,air_yards_share,fantasy_points,fantasy_points_ppr,headshot_url
0,Jordan Love,passing_yards,252.5,251.5,249.5,GB,PHI,1.0,17.0,34.0,260.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,16.40,16.40,https://static.www.nfl.com/image/upload/f_auto...
1,Jordan Love,passing_yards,252.5,251.5,249.5,GB,MIN,4.0,32.0,54.0,389.0,4.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,28.16,28.16,https://static.www.nfl.com/image/upload/f_auto...


In [9]:
etr = pd.read_csv(r"C:\Users\mikej\Downloads\Weekly Projections Detail (5).csv").fillna(0)
etr.columns = ['player','pos','team','week_opponent','etr_completions','etr_attempts',
               'etr_passing_yards','etr_passing_tds','etr_interceptions','etr_carries',
               'etr_rushing_yards','etr_rushing_tds','etr_receptions','etr_receiving_yards',
               'etr_receiving_tds']
etr = etr.drop(columns=['pos','team','etr_passing_tds','etr_interceptions','etr_rushing_tds','etr_receiving_tds'])
etr.head(2)

,player,week_opponent,etr_completions,etr_attempts,etr_passing_yards,etr_carries,etr_rushing_yards,etr_receptions,etr_receiving_yards
0,Kyler Murray,@MIN,20.4,31.4,218.4,5.1,31.0,0.0,0.0
1,James Conner,@MIN,0.0,0.0,0.0,14.2,59.6,3.1,21.8


In [10]:
# df = merged_books.merge(etr,on='player',how='left')

df = merged_books.copy()

df = df[['player','market','ud_line','pp_line','br_line','team',#'week_opponent',
       'week','opponent_team', 'completions', 'attempts', 'passing_yards',
       'passing_tds', 'carries', 'rushing_yards', 'rushing_tds', 'receptions',
       'targets', 'receiving_yards', 'receiving_tds', 'target_share',
       'air_yards_share', 'fantasy_points', 'fantasy_points_ppr',
       # 'headshot_url', 'etr_completions', 'etr_attempts',
       # 'etr_passing_yards', 'etr_carries', 'etr_rushing_yards',
       # 'etr_receptions', 'etr_receiving_yards'
       ]]
df.head(2)

,player,market,ud_line,pp_line,br_line,team,week,opponent_team,completions,attempts,passing_yards,passing_tds,carries,rushing_yards,rushing_tds,receptions,targets,receiving_yards,receiving_tds,target_share,air_yards_share,fantasy_points,fantasy_points_ppr
0,Jordan Love,passing_yards,252.5,251.5,249.5,GB,1.0,PHI,17.0,34.0,260.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,16.40,16.40
1,Jordan Love,passing_yards,252.5,251.5,249.5,GB,4.0,MIN,32.0,54.0,389.0,4.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,28.16,28.16


In [11]:
spreads = pd.read_csv(r"C:\Users\mikej\Desktop\mean-median\data\game_lines.csv")           #game spreads and totals

spreads.head(2)

,team,home_away,week_opponent,spread,over_under
0,Lions,True,Packers,-3.5,51.5
1,Packers,False,Lions,3.5,51.5


In [12]:
# add long team name
team_full = ['Cardinals','Falcons','Ravens','Bills','Panthers','Bears','Bengals','Browns','Cowboys','Broncos','Lions','Packers','Texans','Colts','Jaguars','Chiefs','Raiders','Chargers','Rams','Dolphins','Vikings','Patriots','Saints','Giants','Jets','Eagles','Steelers','49ers','Seahawks','Buccaneers','Titans','Commanders']
team_abr = ['ARI','ATL','BAL','BUF','CAR','CHI','CIN','CLE','DAL','DEN','DET','GB','HOU','IND','JAX','KC','LV','LAC','LAR','MIA','MIN','NE','NO','NYG','NYJ','PHI','PIT','SF','SEA','TB','TEN','WAS']
zipped = list(zip(team_abr,team_full))
team_dict = dict(zipped)
df['team_long'] = df['team'].map(team_dict)

# add game spreads and totals
zipped = list(zip(spreads.team,spreads.week_opponent))
opponent_dict = dict(zipped)
df['week_opponent'] = df['team_long'].map(opponent_dict)

zipped = list(zip(spreads.team,spreads.spread))
spread_dict = dict(zipped)
df['spread'] = df['team_long'].map(spread_dict)

zipped = list(zip(spreads.team,spreads.over_under))
ou_dict = dict(zipped)
df['over_under'] = df['team_long'].map(ou_dict)

df

,player,market,ud_line,pp_line,br_line,team,week,opponent_team,completions,attempts,passing_yards,passing_tds,carries,rushing_yards,rushing_tds,receptions,targets,receiving_yards,receiving_tds,target_share,air_yards_share,fantasy_points,fantasy_points_ppr,team_long,week_opponent,spread,over_under
0,Jordan Love,passing_yards,252.5,251.5,249.5,GB,1.0,PHI,17.0,34.0,260.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,16.400000,16.400000,Packers,Lions,3.5,51.5
1,Jordan Love,passing_yards,252.5,251.5,249.5,GB,4.0,MIN,32.0,54.0,389.0,4.0,1.0,6.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,28.160000,28.160000,Packers,Lions,3.5,51.5
2,Jordan Love,passing_yards,252.5,251.5,249.5,GB,5.0,LA,15.0,26.0,224.0,2.0,3.0,10.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,15.960000,15.960000,Packers,Lions,3.5,51.5
3,Jordan Love,passing_yards,252.5,251.5,249.5,GB,6.0,ARI,22.0,32.0,258.0,4.0,5.0,13.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,25.620001,25.620001,Packers,Lions,3.5,51.5
4,Jordan Love,passing_yards,252.5,251.5,249.5,GB,7.0,HOU,24.0,33.0,220.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,16.799999,16.799999,Packers,Lions,3.5,51.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5662,Dontayvion Wicks,rush+rec_tds,NaN,0.5,NaN,GB,8.0,JAX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,11.0,0.0,0.074074,0.032374,1.100000,3.100000,Packers,Lions,3.5,51.5
5663,Dontayvion Wicks,rush+rec_tds,NaN,0.5,NaN,GB,9.0,DET,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.081081,0.118280,0.000000,0.000000,Packers,Lions,3.5,51.5
5664,Dontayvion Wicks,rush+rec_tds,NaN,0.5,NaN,GB,11.0,CHI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.058824,0.028169,0.000000,0.000000,Packers,Lions,3.5,51.5
5665,Dontayvion Wicks,rush+rec_tds,NaN,0.5,NaN,GB,12.0,SF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,25.0,0.0,0.090909,0.133094,2.500000,3.500000,Packers,Lions,3.5,51.5


In [13]:
df = df[['player', 'market', 'ud_line', 'pp_line', 'br_line', 'team','team_long',
         'week_opponent',
         'spread','over_under', 
        #  'etr_completions', 'etr_attempts',
        #  'etr_passing_yards', 'etr_carries', 'etr_rushing_yards',
        #  'etr_receptions', 'etr_receiving_yards',
         'week','opponent_team', 'completions', 'attempts',
         'passing_yards', 'passing_tds', 'carries', 'rushing_yards',
         'rushing_tds', 'receptions', 'targets', 'receiving_yards',
         'receiving_tds', 'target_share', 'air_yards_share', 'fantasy_points',
         'fantasy_points_ppr', 
        #  'headshot_url'
         ]]

In [27]:
def pp_delta(row):
    if row['market'] == 'receiving_yards':
        return row['receiving_yards'] - row['pp_line']
    elif row['market'] == 'rushing_yards':
        return row['rushing_yards'] - row['pp_line']
    elif row['market'] == 'fantasy_points':
        return row['fantasy_points'] - row['pp_line']
    elif row['market'] == 'targets':
        return row['targets'] - row['pp_line']
    elif row['market'] == 'receptions':
        return row['receptions'] - row['pp_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['pp_line']
    elif row['market'] == 'completions':
        return row['completions'] - row['pp_line']
    elif row['market'] == 'passing_tds':
        return row['passing_tds'] - row['pp_line']
    elif row['market'] == 'passing_yards':
        return row['passing_yards'] - row['pp_line']
    # elif row['market'] == 'sacks':
    #     return row['sacks'] - row['pp_line']
    elif row['market'] == 'rushing_tds':
        return row['rushing_tds'] - row['pp_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['pp_line']
    else:
        return np.nan
    
def ud_delta(row):
    if row['market'] == 'receiving_yards':
        return row['receiving_yards'] - row['ud_line']
    elif row['market'] == 'rushing_yards':
        return row['rushing_yards'] - row['ud_line']
    elif row['market'] == 'fantasy_points':
        return row['fantasy_points'] - row['ud_line']
    elif row['market'] == 'targets':
        return row['targets'] - row['ud_line']
    elif row['market'] == 'receptions':
        return row['receptions'] - row['ud_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['ud_line']
    elif row['market'] == 'completions':
        return row['completions'] - row['ud_line']
    elif row['market'] == 'passing_tds':
        return row['passing_tds'] - row['ud_line']
    elif row['market'] == 'passing_yards':
        return row['passing_yards'] - row['ud_line']
    # elif row['market'] == 'sacks':
    #     return row['sacks'] - row['ud_line']
    elif row['market'] == 'rushing_tds':
        return row['rushing_tds'] - row['ud_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['ud_line']
    else:
        return np.nan
    
def br_delta(row):
    if row['market'] == 'receiving_yards':
        return row['receiving_yards'] - row['br_line']
    elif row['market'] == 'rushing_yards':
        return row['rushing_yards'] - row['br_line']
    elif row['market'] == 'fantasy_points':
        return row['fantasy_points'] - row['br_line']
    elif row['market'] == 'targets':
        return row['targets'] - row['br_line']
    elif row['market'] == 'receptions':
        return row['receptions'] - row['br_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['br_line']
    elif row['market'] == 'completions':
        return row['completions'] - row['br_line']
    elif row['market'] == 'passing_tds':
        return row['passing_tds'] - row['br_line']
    elif row['market'] == 'passing_yards':
        return row['passing_yards'] - row['br_line']
    # elif row['market'] == 'sacks':
    #     return row['sacks'] - row['ud_line']
    elif row['market'] == 'rushing_tds':
        return row['rushing_tds'] - row['br_line']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['br_line']
    else:
        return np.nan
    
df['pp_delta'] = df.apply(lambda row: pp_delta(row), axis=1)
df['ud_delta'] = df.apply(lambda row: ud_delta(row), axis=1)  
df['br_delta'] = df.apply(lambda row: br_delta(row), axis=1)


C:\Users\mikej\AppData\Local\Temp\ipykernel_24704\3793981611.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pp_delta'] = df.apply(lambda row: pp_delta(row), axis=1)
C:\Users\mikej\AppData\Local\Temp\ipykernel_24704\3793981611.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ud_delta'] = df.apply(lambda row: ud_delta(row), axis=1)
C:\Users\mikej\AppData\Local\Temp\ipykernel_24704\3793981611.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [28]:
# get etr_proj deltas

def etr_delta(row):
    if row['market'] == 'receiving_yards':
        return row['receiving_yards'] - row['etr_receiving_yards']
    elif row['market'] == 'rushing_yards':
        return row['rushing_yards'] - row['etr_rushing_yards']
    # elif row['market'] == 'fantasy_points':
    #     return row['fantasy_points'] - row['etr_proj']
    # elif row['market'] == 'targets':
    #     return row['targets'] - row['etr_proj']
    elif row['market'] == 'receptions':
        return row['receptions'] - row['etr_receptions']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['etr_attempts']
    elif row['market'] == 'completions':
        return row['completions'] - row['etr_completions']
    # elif row['market'] == 'passing_tds':
    #     return row['passing_tds'] - row['etr_proj']
    elif row['market'] == 'passing_yards':
        return row['passing_yards'] - row['etr_passing_yards']
    # elif row['market'] == 'sacks':
    #     return row['sacks'] - row['etr_proj']
    # elif row['market'] == 'rushing_tds':
    #     return row['rushing_tds'] - row['etr_proj']
    elif row['market'] == 'attempts':
        return row['attempts'] - row['etr_attempts']
    else:
        return np.nan

    
df['etr_delta'] = df.apply(lambda row: etr_delta(row), axis=1)  

KeyError: 'etr_passing_yards'

In [14]:
df.to_csv(r"C:\Users\mikej\Desktop\mean-median\data\mean_median.csv",index=False)